Reference : https://github.com/entbappy/NLP-Projects-Notebooks/blob/master/En_Hi_language_translation.ipynb

In [1]:
#GPU check

!nvidia-smi

Sat Mar 16 01:24:56 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.61                 Driver Version: 551.61         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   52C    P3             11W /   35W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu -q --user
!pip install sacremoses --user
!pip install moviepy 
!pip install vosk
!pip install pydub


In [3]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Audio_ext:

In [4]:
import moviepy.editor 
cvt_video=moviepy.editor.VideoFileClip(r"C:\Users\ASUS\Desktop\STT\RIISE 2022 final video.mp4")
audio_clip = cvt_video.audio
audio_path=r"C:\Users\ASUS\Desktop\STT\audio_extract.mp3"
audio_clip.write_audiofile(audio_path, codec='mp3')
audio_clip.close()

MoviePy - Writing audio in C:\Users\ASUS\Desktop\STT\audio_extract.mp3


MoviePy - Done.


Speech to text

In [5]:
from vosk import Model, KaldiRecognizer
import wave
from pydub import AudioSegment

model_path = r"C:\Users\ASUS\Desktop\STT\vosk-model-small-en-us-0.15"
model = Model(model_path)
recognizer = KaldiRecognizer(model, 16000)

audio_file_path = r"C:\Users\ASUS\Desktop\STT\audio_extract.mp3"
audio = AudioSegment.from_mp3(audio_file_path)

wav_audio_file_path = "audio_extract.wav"


audio.export(wav_audio_file_path, format="wav", parameters=["-ac", "1", "-ar", "16000"])

with wave.open(wav_audio_file_path, 'rb') as wave_file:
    while True:
        data = wave_file.readframes(4096)
        
        if len(data) == 0:
            break
        recognizer.AcceptWaveform(data)

result = recognizer.FinalResult()



In [6]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

Dataset:

In [7]:
raw_datasets = load_dataset("cfilt/iitb-english-hindi")

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [9]:
raw_datasets['train'][1]

{'translation': {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}}

Data preprocessing (Tokenizing)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
tokenizer("Hello this is a good sentance!")

{'input_ids': [12110, 90, 23, 19, 229, 374, 5418, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
tokenizer(["Hello this is nice", "This is another nice"])

{'input_ids': [[12110, 90, 23, 10715, 0], [239, 23, 414, 10715, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [13]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["एक्सेर्साइसर पहुंचनीयता अन्वेषक"]))

{'input_ids': [[26618, 16155, 346, 33383, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


In [14]:
max_input_length = 4096
max_target_length = 4096

source_lang = "en"
target_lang = "hi"


def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [16]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map: 100%|██████████| 2507/2507 [00:00<00:00, 5854.07 examples/s]


In [17]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [18]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [20]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=4096)

In [21]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [22]:

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [23]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)


In [24]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [25]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


156/156 [==============================] - 309s 2s/step - loss: 3.7638 - val_loss: 3.9493


In [26]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]], 'forced_eos_token_id': 0}


Testing:

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [28]:
input_text = result

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=4096)
print(out)

tf.Tensor(
[[61949 13231    83 17581   224     6  1398     2    83   629   104  1776
     11    89   207    11   753     6   207    11   176   672  1066  1291
      6    39 17487   254    25    89  3376     6   207    11   104    65
     57   153   254     6   207    11    99    18  1291     6    39 17487
    254     2    41    25    89  3376     6   207    11   176    58  7869
     28     2    41    25    89   207    11    99     6   207    11   176
     58  7869  1066     5     9    41    25    89  3376     6   207    11
     99     6   207    11   176   672  1066     5     2   992    88    89
   3376     6   207    11   176    58  7869  1066     6   207    11   176
    672  1066     5     2   992    75    99     6   207    11   632    28
     25    89  3376     6   207    11    65    57   160    28     2   992
     88    89  3376     6   207    11   176   672  1066     6   207    11
     99     6   207    11   176   672  1066     6   207    11   176   672
   1066     5     2   992  

In [29]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

{ "पाठ" के अलावा, "और मैं वास्तव में इस बारे में सूचना के बारे में बहुत अधिक जानकारी देखने के लिए उत्सुक हूँ कि इस कार्यक्रम के बारे में मैं क्या कर रहा हूँ के बारे में लोगों को देखने के लिए उत्सुक हूँ, जो कि इस कार्यक्रम के बारे में बहुत ही दिलचस्प हैं, जो कि इस बारे में लोगों के बारे में बहुत ही दिलचस्प जानकारी है और जो कि इस कार्यक्रम के बारे में लोगों के बारे में बहुत अधिक जानकारी है, जबकि वे इस कार्यक्रम के बारे में बहुत ही दिलचस्प जानकारी के बारे में बहुत अधिक जानकारी है, जबकि हम लोगों के बारे में जानते हैं कि इस कार्यक्रम के बारे में क्या कर रहे हैं, जबकि वे इस कार्यक्रम के बारे में बहुत अधिक जानकारी के बारे में लोगों के बारे में बहुत अधिक जानकारी के बारे में बहुत अधिक जानकारी है, जबकि वे इस कार्यक्रम के बारे में लोगों के बारे में बहुत अधिक जानकारी है कि वे इस कार्यक्रम के बारे में क्या कर रहे हैं।
